## Create Domain Level Features

### Requirements
1. Dictionary files for each domain
2. List of domains
3. Domain stats dataframe
4. BLOSUM62 dictionary
5. pfam emissions probability dictionary
6. Fixed threshold clustering csv
7. Percentile clustering csv

### Instructions
Run cells in order.

### Output
A csv file with rows labelled by domain name and columns by feature.

In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle
import datetime
import math
import sys
curr_dir = !pwd
sys.path.append(curr_dir[0] + "/../5.HMM_alter_align") 
from calc_exac_freq_func import codon_table
from dnds_func import calculate_ns, seq_ns
from collections import defaultdict
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [2]:
#Getting path
curr_dir = !pwd
intance_cutoff = "50"

#Reading the list of filtered domains
with open(curr_dir[0]+"/../5.domains_stats/filtered"+intance_cutoff+"_list.pik", 'rb') as handle:
    filtered_domains_list = pickle.load(handle)
filtered_domains_list.sort()

#Reading the table of all domains stats
filtered_domains_df = pd.read_csv(curr_dir[0]+"/../5.domains_stats/filtered"+intance_cutoff+"_domains_df.csv", sep='\t', index_col=0)

#Read the substitutions table (for the dN/dS calculation)
with open(curr_dir[0]+"/codon_ns_table.pik", 'rb') as handle:
    codon_ns_table = pickle.load(handle)

#Reading the BLOSUM62 dict
with open(curr_dir[0]+"/../BLOSUM62/BLOSUM62_dict.pik", 'rb') as handle:
    blosum62_dict = pickle.load(handle)
    
#Reading the HMM dict
with open(curr_dir[0]+"/../2.parse_Pfam/v30/domains_hmm_prob_dict.pik", 'rb') as handle:
    hmm_prob_dict = pickle.load(handle)
    
#Reading clustering files
clustering_thresh = pd.read_csv(curr_dir[0]+"/clustering_fixedThresh.csv", sep=',', index_col=0)
clustering_percentile = pd.read_csv(curr_dir[0]+"/clustering_percentile.csv", sep=',', index_col=0)

#### Functions used in computation of features

In [3]:
# Calculates a normalized Shannon entropy (from Miller et al, 2015)
def entropy(a):
    a = np.asarray(a) / sum(a)
    entropy = 0
    for val in a:
        if val == 0 or np.isnan(val):
            continue
        entropy += val * math.log(val)
    return(-entropy / math.log(len(a)))

# Measure density of vector – still experimental
def density(a,window):
    norm = []
    for i in range(0,len(a)):
        norm.append(float(a[i])/sum(a))
    sums = np.empty(0)
    for i in range(0,len(a)+window-1):
        sums = np.append(sums,sum(norm[max(0,i-window+1):min(len(norm),i+1)]))
    return(entropy(sums))

#### Calculate domain features

In [45]:
input_path = curr_dir[0]+"/../5.HMM_alter_align/domains_states_dicts/pfam-v30/"
features_dict = defaultdict(list)

for domain_name in filtered_domains_list:
    
    #Reading the domain states dictionary
    domain_dirfiles = !ls -t $input_path$domain_name
    #Find the most recent file
    recent_priority = -1
    recent_filename = ""
    for f in domain_dirfiles:
        tokens = f.split("_")
        date = tokens[len(tokens)-1].split(".")
        month = int(date[0])
        day = int(date[1])
        #Not all files have years, but those that do are the most recent
        if date[2] != "pik":
            year = int(date[2])
        else:
            year = 0
        priority = year*1000 + month*50 + day
        if priority > recent_priority:
            recent_priority = priority
            recent_filename = f
    with open(input_path+domain_name+"/"+recent_filename, 'rb') as handle:
        states_dict = pickle.load(handle)
        
    #Initializing feature counters
    maf_sum = 0
    sites_aa_num = 0
    sites_aa_alter_num = 0
    sites_snp_alter_num = 0
    sites_poly_aa_num = 0 #The number of different aa in all the altered sites (most are 1)
    sites_poly_aa_several = 0
    
    #Rare-poly-counters
    maft_5 =  0.005
    maft_05 = 0.0005
    maft_005 = 0.00005
    rare_5_num = 0
    rare_05_num = 0
    rare_005_num = 0
    
    #BLOSUM62_vals
    blosum62_list = []
    weigted_blosum62_list = []
    
    #SIFT counters
    sift_sum = 0
    sift_cnt = 0
    
    #PolyPhen counters
    polyphen_sum = 0
    polyphen_cnt = 0
    
    #dn/ds counters and variables
    ref_seq = ""
    #ref_af_list = []
    Nd = 0
    Sd = 0
    
    #Entropy
    nonsyn_by_pos = {}
    nonsyn_by_gene = {}
    
    #Conservation scores
    phyloP_cutoff = 1.31
    phastCons_con_cutoff = 0.95
    phastCons_noncon_cutoff = 0.05
    counter_phyloP = 0
    counter_phastCons = 0
    sum_phyloP = 0
    sum_phastCons = 0
    num_con_phyloP = 0
    num_con_phastCons = 0
    num_noncon_phyloP = 0
    num_noncon_phastCons = 0
    
    #Nonsyn mutations in conserved aa positions
    num_con_nonsyn = 0
    
    #Very common mutations
    common_cutoff = 0.1
    num_common = 0
    
    #pfam conserved sites
    num_conserved = 0
    sum_max = 0
    prob_entropy = []
    total_num = len(hmm_prob_dict[domain_name].keys())
    con_threshold = 0.8
    for state in hmm_prob_dict[domain_name].keys():
        prob_list = hmm_prob_dict[domain_name][state]
        prob_entropy.append(entropy(prob_list))
        sum_max += max(prob_list)
        if (max(prob_list) > con_threshold):
            num_conserved += 1
    
    for state in states_dict:
        sites_aa_num += len(states_dict[state])
        for d in states_dict[state]:
            #Creating a position pseudo-ref sequence
            ref_codon = d["bp_ref"]
            ref_seq = ref_seq+ref_codon
            
            #Calculating frequency-based N/S
            bp_af_adj_dict = d["bp_af_adj_dict"]
            for alt_codon in bp_af_adj_dict.keys():
                alt_aa = codon_table[alt_codon]
                #syn
                if (alt_aa == d["aa_ref"]):
                    Sd += bp_af_adj_dict[alt_codon]
                #Non-syn
                else:
                    Nd += bp_af_adj_dict[alt_codon]
                    
            #Conservation
            for score in d["phyloP"]:
                counter_phyloP += 1
                sum_phyloP += score
                if score > phyloP_cutoff:
                    num_con_phyloP += 1
                elif score < -phyloP_cutoff:
                    num_noncon_phyloP += 1
            for score in d["phastCons"]:
                counter_phastCons += 1
                sum_phastCons += score
                if score > phastCons_con_cutoff:
                    num_con_phastCons += 1
                elif score < phastCons_noncon_cutoff:
                    num_noncon_phastCons += 1
            
            if (d["af_adj"] > 0):
                sites_aa_alter_num += 1
                sites_snp_alter_num += len(d["an_adj"])
                maf_sum += d["af_adj"]
                
                #Number of different polymorphisms at this site
                site_poly_num = len(d["alterations_af_adj_dict"].keys())
                sites_poly_aa_num += site_poly_num
                if (site_poly_num > 1):
                    sites_poly_aa_several += 1
                
                #Rare poly features
                if (d["af_adj"] < maft_005):
                    rare_005_num += 1
                    rare_05_num += 1
                    rare_5_num += 1
                elif (d["af_adj"] < maft_05):
                    rare_05_num += 1
                    rare_5_num += 1
                elif (d["af_adj"] < maft_5):
                    rare_5_num += 1
                
                #Common SNP
                if(d["af_adj"] > common_cutoff):
                    num_common += 1
                    
                #BLOSUM62 features
                ref = d["aa_ref"]
                for alt in d["alterations_af_adj_dict"].keys():
                    blosum_val = blosum62_dict[ref][alt]
                    af_adj = np.mean(d["alterations_af_adj_dict"][alt])
                    blosum62_list.append(blosum_val)
                    weigted_blosum62_list.append(blosum_val*af_adj)
                
                #SIFT
                sift_list = d["SIFT"]
                for s in sift_list:
                    if (s != ""):
                        sift_sum += float(s[s.find("(")+1:s.find(")")])
                        sift_cnt += 1
                        
                #PolyPhen
                polyphen_list = d["PolyPhen"]      
                for s in polyphen_list:
                    if (s != ""):
                        polyphen_sum += float(s[s.find("(")+1:s.find(")")])
                        polyphen_cnt += 1
                
                #Entropy
                #Position
                if state in nonsyn_by_pos.keys():
                    nonsyn_by_pos[state].append(d["af_adj"])
                else:
                    nonsyn_by_pos[state] = [d["af_adj"]]
                #Gene
                key = d['ens_gene']
                if key in nonsyn_by_gene.keys():
                    nonsyn_by_gene[key].append(d["af_adj"])
                else:
                    nonsyn_by_gene[key] = [d["af_adj"]]

                #Nonsyn conserved mutations
                if len(d["phyloP"]) == 3 and np.mean(d["phyloP"]) > phyloP_cutoff:
                    num_con_nonsyn += 1
        
    #Feature: domain length
    domain_len = len(states_dict.keys())
    features_dict[domain_name].append(domain_len)
    
    #Feature: average MAF overall aa sites
    avg_maf_overall = maf_sum/float(sites_aa_num)
    features_dict[domain_name].append(avg_maf_overall)
    
    #Feature: average MAF of all the altered sites
    avg_maf_only_altered = maf_sum/float(sites_aa_alter_num)
    features_dict[domain_name].append(avg_maf_only_altered)
    
    #Feature: number of alterations - aa level (raw and normalized by domain length)
    norm_aa_alter_num = sites_aa_alter_num/float(domain_len)
    features_dict[domain_name].append(sites_aa_alter_num)
    features_dict[domain_name].append(norm_aa_alter_num)
    
    #Feature: number of alterations - DNA level (raw and normalized by domain length)
    norm_snp_alter_num = sites_snp_alter_num/float(domain_len)
    features_dict[domain_name].append(sites_snp_alter_num)
    features_dict[domain_name].append(norm_snp_alter_num)
    
    #Feature: fraction of aa alterations (fraction of non-zero alterations)
    frac_alter_aa = sites_aa_alter_num/float(sites_aa_num)
    features_dict[domain_name].append(frac_alter_aa)
    
    #Feature: average number of polymorphisms at one site
    avg_poly_aa = sites_poly_aa_num/float(sites_aa_alter_num)
    features_dict[domain_name].append(avg_poly_aa)
    
    #Feature: fraction of altered sites with more than 1 polymorphism
    frac_poly_several = sites_poly_aa_several/float(sites_aa_alter_num)
    features_dict[domain_name].append(frac_poly_several)
    
    #Feature: fraction of rare SNPs (0.5%)
    frac_rare_5 = rare_5_num/float(sites_aa_alter_num)
    features_dict[domain_name].append(frac_rare_5)
    
    #Feature: fraction of rare SNPs (0.05%)
    frac_rare_05 = rare_05_num/float(sites_aa_alter_num)
    features_dict[domain_name].append(frac_rare_05)
    
    #Feature: fraction of rare SNPs (0.005%)
    frac_rare_005 = rare_005_num/float(sites_aa_alter_num)
    features_dict[domain_name].append(frac_rare_005)
    
    #Feature: BLOSUM62 average
    blosum62_avg = sum(blosum62_list)/float(len(blosum62_list))
    weigted_blosum62_avg = sum(weigted_blosum62_list)/float(len(weigted_blosum62_list))
    features_dict[domain_name].append(blosum62_avg)
    features_dict[domain_name].append(weigted_blosum62_avg)
    
    #Feature: pseudo-sequence dN/dS        
    (N,S) = seq_ns(ref_seq) #Refrence expected syn/nonsyn per site
    if (N == 0): 
        PN = 0
    else:
        PN = Nd/float(N) #Proportion of nonsyn
    if (S == 0):
        PS = 0
    else:
        PS = Sd/float(S) #Proportion of syn

    #num of nonsyn substitutions per syn site
    dN = -0.75 * (np.log(1-4*PN/float(3)))
    #num of syn substitutions per nonsyn site
    dS = -0.75 * (np.log(1-4*PS/float(3)))

    if (dN ==0 or dS == 0):
        dN_dS = 1 #There isn't enough information to calculate dN/dS
    else:
        dN_dS = dN/dS

    features_dict[domain_name].append(dN_dS)
    
    #Feature: SIFT average
    if (sift_cnt > 0):
        sift_avg = sift_sum/float(sift_cnt)
    else:
        sift_avg = -1
    features_dict[domain_name].append(sift_avg)
    
    #Feature: PolyPhen average
    if (polyphen_cnt > 0):
        polyphen_avg = polyphen_sum/float(polyphen_cnt)
    else:
        polyphen_avg = -1
    features_dict[domain_name].append(polyphen_avg)
    
    #Feature: Fraction of DNA sites altered
    frac_snp_alter = float(sites_snp_alter_num) / (3*sites_aa_num)
    features_dict[domain_name].append(frac_snp_alter)
    
    #Case where there are no mutations for a position is handled, so no need to throw anything
    np.seterr(divide='ignore',invalid='ignore')
    
    #Feature: Entropy by position
    avg_nonsyn_pos = np.zeros(len(nonsyn_by_pos.keys()))
    index = 0
    for key in nonsyn_by_pos.keys():
        avg_nonsyn_pos[index] = np.median(nonsyn_by_pos[key])
        index += 1
    features_dict[domain_name].append(entropy(avg_nonsyn_pos))
    
    #Feature: Windowed entropy by position
    features_dict[domain_name].append(density(avg_nonsyn_pos,20))
    
    #Feature: Entropy by gene
    avg_nonsyn_gene = np.zeros(len(nonsyn_by_gene.keys()))
    index = 0
    for key in nonsyn_by_gene.keys():
        avg_nonsyn_gene[index] = np.median(nonsyn_by_gene[key])
        index += 1
    features_dict[domain_name].append(entropy(avg_nonsyn_gene))
    
    #Reset warnings
    np.seterr(divide='warn',invalid='warn')
    
    #Feature: Average phyloP
    avg_phyloP = sum_phyloP / counter_phyloP
    features_dict[domain_name].append(avg_phyloP)
    
    #Feature: Average phastCons
    avg_phastCons = sum_phastCons / counter_phastCons
    features_dict[domain_name].append(avg_phastCons)
    
    #Feature: Ratio phyloP
    ratio_phyloP = float(num_con_phyloP) / num_noncon_phyloP
    features_dict[domain_name].append(ratio_phyloP)
    
    #Feature: Ratio phastCons
    ratio_phastCons = float(num_con_phastCons) / num_noncon_phastCons
    features_dict[domain_name].append(ratio_phastCons)
    
    #Feature: pfam emission prob fraction
    frac_conserved = float(num_conserved) / total_num
    features_dict[domain_name].append(frac_conserved)
    
    #Feature: pfam emission prob average max
    avg_max = sum_max / total_num
    features_dict[domain_name].append(avg_max)
    
    #Feature: Average pfam emission prob entropy
    features_dict[domain_name].append(np.mean(prob_entropy))
    
    #Feature: Clustering with 0.005% cutoff
    features_dict[domain_name].append(clustering_thresh.loc[domain_name,"5e-05"])
    
    #Feature: Clustering with 90th percentile
    features_dict[domain_name].append(clustering_percentile.loc[domain_name,"90"])
    
    #Feature: Fraction of nonsyn altered positions that are conserved
    frac_con = float(num_con_nonsyn) / sites_aa_alter_num
    features_dict[domain_name].append(frac_con)
    
    #Feature: Common SNPs
    frac_common = float(num_common) / sites_aa_alter_num
    features_dict[domain_name].append(frac_common)
    
    print("Finished domain "+domain_name)

Finished domain 7TM_GPCR_Srsx
Finished domain 7tm_1
Finished domain 7tm_4
Finished domain AAA
Finished domain AAA_5
Finished domain ABC_membrane
Finished domain ABC_tran
Finished domain Ank
Finished domain Ank_2
Finished domain Ank_3
Finished domain Ank_4
Finished domain Ank_5
Finished domain Annexin
Finished domain Arf
Finished domain Arm
Finished domain BACK
Finished domain BTB
Finished domain BTB_2
Finished domain Bromodomain
Finished domain C1-set
Finished domain C1_1
Finished domain C2
Finished domain C2-set_2
Finished domain CBFD_NFYB_HMF
Finished domain CH
Finished domain CUB
Finished domain Cadherin
Finished domain Cadherin_2
Finished domain Cadherin_3
Finished domain Cadherin_C_2
Finished domain Cadherin_tail
Finished domain Collagen
Finished domain DEAD
Finished domain DUF1220
Finished domain EF-hand_1
Finished domain EF-hand_5
Finished domain EF-hand_6
Finished domain EF-hand_7
Finished domain EF-hand_8
Finished domain EGF
Finished domain EGF_2
Finished domain EGF_3
Finished

#### Export features to a well-formatted data frame

In [46]:
domains_features_df = pd.DataFrame.from_dict(features_dict,orient='index')
domains_features_df.columns = ["length", "avg_maf_all", "avg_maf_altered", "alter_num_aa", "alter_num_aa_norm", 
                               "alter_num_dna", "alter_num_dna_norm", "frac_alter_aa", "avg_poly", "frac_poly_several", 
                               "rare_poly_0.5%", "rare_poly_0.05%", "rare_poly_0.005%", "BLOSUM_avg", "weighted_BLOSUM_avg", 
                               "pseudo_dNdS", "SIFT", "PolyPhen","frac_alter_dna", "entropy_pos_nonsyn", "entropy_nonsyn_window",
                               "entropy_gene_nonsyn", "avg_phyloP", "avg_phastCons", "ratio_phyloP", "ratio_phastCons",
                               "frac_aa_conserved", "avg_max_aa_conservation", "pfam_prob_entropy", "clustering_0.005%",
                               "clustering_90", "frac_aa_nonsyn_conserved", "frac_common_poly"]
domains_features_df = domains_features_df.sort_index()

#Adding the data from the df
domains_features_df["num_genes"] = filtered_domains_df["genes"]
domains_features_df["num_instances"] = filtered_domains_df["instances"]

#Computing log2 of genes number
domains_features_df["num_genes_log2"] = domains_features_df["num_genes"].apply(lambda x: np.log2(x))
domains_features_df["num_instances_log2"] = domains_features_df["num_instances"].apply(lambda x: np.log2(x))

#Save to file
domains_features_df.to_csv(curr_dir[0]+"/domains_features_df_filtered"+intance_cutoff+".csv", sep='\t')